# Partie 1. - Découverte du jeu de données et lecture des documents

Dans cette partie, nous allons travailler sur un jeu de données que nous pourrions avoir en entreprise. 

Ce jeu de données contient un ensemble de documents PDF (majoritaires), word et PowerPoint dont certains sont des documents scannés.

L'objectif de cette partie est de construire un programme informatique permettant, automatiquement, d'ouvrir les documents et d'en extraire les contenus. Pour les documents scannés ou les images, nous utilisons la technique d'OCR (Object Character Recognition) pour extraire le texte des images.

Les bibliothèques que nous allons utiliser sont les suivantes :
- **Apache Tika** (https://tika.apache.org/) : lecture d'un grand nombre de format de documents et extraction du contenu et des métadonnées
- **Tesseract** (https://github.com/tesseract-ocr/tesseract) : moteur de reconnaissance optique de caractères (OCR)

## Import des bibliothèques logicielles et configuration

In [ ]:
from tika import parser # Bibliothèque Tika pour la lecture des documents

# Autres bibliothèques utilitaires pour le TP
import os, glob, random, tabulate
from IPython.display import Image
from  IPython.core.display import display, HTML

In [ ]:
DATASET_DIRECTORY = "./dataset" # indique le répertoire de base où se trouve les document à indexer
USE_OCR = True # indique si l'OCR doit être activé ou non (ralenti le processus)

## Téléchargement du jeu de données

Exécuter la commande ci-dessous pour télécharger le jeu de données provenant de `data.gouv.fr`.

In [ ]:
!wget https://static.data.gouv.fr/resources/texte-provenant-des-pdfs-trouves-sur-data-gouv-fr/20200520-003700/data-gouv-pdf-txt.tgz
!tar -zxvf /workspace/data-gouv-pdf-txt.tgz --directory /workspace/dataset/
!rm /workspace/data-gouv-pdf-txt.tgz

Explorer le jeu de données en allant dans le dossier `dataset` depuis Jupyter (http://127.0.0.1:8889).

## 1. Lecture des documents et extraction du texte

Dans cette section, nous allons parcourir l'ensemble des documents du dataset puis pour chacun d'entre eux, nous allons extraire :
- son titre (nom de fichier)
- son contenu ;
- son chemin sur le système de fichiers.

Le bloc de code ci-dessous est une fonction permettant de parcourir l'ensemble des fichiers présent dans un dossier et retourne les chemins de ces derniers.

Exécuter le bloc de code ci-dessous pour charger cette fonction afin de l'utiliser par la suite.

In [ ]:
def get_all_files_in_directory(directory):
    """Fonction retournant l'ensemble des chemins des fichiers se trouvant dans le répertoire directory."""
    file_paths = set()
    for filename in glob.iglob(directory + '/**', recursive=True):
        if os.path.isfile(filename): 
            file_paths.add(filename)
    return file_paths

def read_files_and_extract_content(input_file_paths):
    """
    Fonction qui permet de lire l'ensemble des fichiers dont les chemins sont spécifiés en paramètre.
    Retourne un dictionnaire contenant les documents extraits. 
    Pour chaque document 4 champs sont récupérés ou calculés : title, path, content, file_size.
    """
    documents = {}
    for input_file_path in input_file_paths:
        parsed_file = parser.from_file(input_file_path)
        title = os.path.basename(input_file_path)
        content = parsed_file['content']
        if(content is None):
            content = "" 
        file_size = os.path.getsize(input_file_path)
        document = {"title": title, "path":input_file_path, "content":content, "file_size": file_size}
        documents[input_file_path] = document
    return documents

Exécuter le bloc de code ci-dessous pour charger un échantillon de 5 chemins de fichiers. Nous chargerons l'ensemble de données complet à la fin de cette partie.

In [ ]:
input_file_paths = get_all_files_in_directory(DATASET_DIRECTORY) # Appel de la fonction et récupération des chemins des fichiers.
input_file_paths_sample = random.sample(input_file_paths, 5)
print("Exemple aléatoires de chemins de fichiers.")
for input_file_path in input_file_paths_sample:
    print("\t" + input_file_path) # Tips : \t permet d'ajouter une tabulation.

Exécuter le bloc de code ci-dessous pour lire l'ensemble des fichiers à partir d'une liste de chemin de fichiers.

_La première exécution peut prendre quelques minutes car il y a téléchargement du serveur Tika_.

In [ ]:
documents = read_files_and_extract_content(input_file_paths)

A cette étape, les documents ont été lus et leurs contenus a été placé dans le dictionnaire `documents` dont les clefs sont des chemins sur le système de fichier et les valeurs les documents extraits. 

Affichons un exemple de document. Les documents sont structurés sous la forme de documents JSON.

In [ ]:
documents['./dataset/data_gouv_txt/Caisse_nationale_de_lassurance_maladie/53699271a3a729239d203f1c--8b29ab5f-173f-45f4-b842-f7ac5213eef5.txt']

### 2. Reconnaissance Optique de Caractères

La reconnaissance optique de caractères (Object Character Recognition - OCR) permet d'extraire du texte à partir d'une image, par exemple un document scanné. 

Cette tâche, simple pour l'humain, l'est moins pour la machine qui ne "voit" que des pixels, parfois bruités avec des tailles de polices différentes. Les algorithmes mis en jeu pour cette tâche procèdent généralement de la façon suivante :
- **Pré-traitement de l'image** pour la normaliser, la débruiter, accentuer les contours de formes, ...
- **Détection des lignes, des mots et des caractères** : seln la qualité de l'image, le texte n'est pas toujours positionnné de manière parfaitement horizontale et les mots et les lignes ne sont pas toujours clairement séparés.
- **Reconnaissance de caractères** : le procédé peut mettre en jeu plusieurs algorithmes, par exemple pour déterminer le mot le plus probable en fonction du contexte. C'est particulièrement utile lorsque les caractères sont semblables, comme 0 (zéro) et O (lettre).
- **Production du texte de sortie.**

Pour réaliser cette tâche, nous pouvons utiliser la célèbre bibliothèque open-source _Tesseract_, aujourd'hui maintenue par Google (https://github.com/tesseract-ocr/tesseract). Tesseract est nativement intégré à Tika qui l'utilise pour extraire du texte lorsque le fichier est un image ou en contient une (dans un PDF par exemple).

L'OCR requiert de la puissance de calcul et peut ralentir les traitements. Aussi, dans le cadre de ce TP, nous n'exploitons que quelques fichiers de type image.

Ci-dessous, nous visualisons une image et le résultat obtenu après OCR.

In [ ]:
img = Image(filename='./dataset/images/ocr-1.png', height=450) 
content = "".join([s for s in documents['./dataset/images/ocr-1.png']["content"].splitlines(True) if s.strip("\r\n")])

display(img)
display(HTML("<h3> Résultat de l'extraction (extraction) </h3>" + content.replace("\n", "<br/>")))

**Analyse**

Nous constatons que le texte est bien reconnu, hormis une virgule qui est détectée comme un point à la 9ième ligne.

Cette technique rend des documents scannés traitables pour des tâches de TALN et cela peut faire gagner énormément de temps lorsque l'on a des milliers de documents.

### 3. Affichage de statistiques sur le jeu de données

**Exercice** : maintenant que nous avons lu et chargé le jeu de données, nous pouvons afficher quelques statistiques. Compléter les lignes marquées **TODO** pour calculer ces statistiques.

In [ ]:
doc_count = 0 # TODO : mettre le code nécessaire pour afficher le nombre de documents. Astuce : utiliser la fonction `len()`
size_on_disk_mo = sum([doc['file_size'] for doc in documents.values()]) / 1000000
average_document_size_ko = size_on_disk_mo / doc_count * 1000
average_chars_per_document = sum([len(doc['content']) for doc in documents.values()]) / doc_count

print("Nombre de documents: {}".format(doc_count))
print("Poids sur disque du jeu de données en Mo: {} Mo".format(size_on_disk_mo))
print("Poids moyen d'un document en Ko: {} Ko".format(average_document_size_ko))
print("Nombre moyen de caractères par document: {}".format(average_chars_per_document))


### 4. Sauvegarde
L'étape de parcours des documents et d'extraction du texte, éventuellement avec l'OCR, peut être très longue.
Aussi, nous allons sauvegarder le résultat de l'extraction au format JSON sur le disque afin de le réutiliser dans la suite du TP.

**Attention** : le fichier produit pèse plusieurs centaine de Mo. L'ouvrir avec un éditeur de texte risque de faire planter la machine.

In [ ]:
f = open("./dataset_processed/dataset.json", "w")
f.write(str(documents))
f.close()

## Conclusion

Dans cette partie, nous avons vu comment lire et extraire le texte de documents bureautiques. D'autres techniques et bibliothèques existent pour extraire de l'information d'autres sources de données (base de données, pages web, ...). Cela n'est pas encore du TALN mais cette étape est nécessaire pour ensuite pouvoir traiter et analyser sereinement les données. 